In [ ]:
# 1.각 compound 들의 조합gene들이
# 2. ppi network로 인하여 seed node로 설정되는데 
# 이 seed 노드는 조합마다 다를 것 같습니다.
# 3. 조합별 Seed node와 위 gene-Phenotype 의 교집합을 phenotype 별로 출력되게 주시면 될 것 같습니다 

In [1]:
# rwr 조화평균 완료

# herb 최종
from tqdm import tqdm_notebook
import numpy as np
import pickle
import pandas as pd
import configparser
import psycopg2
import os
import ast
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
import random
import time
import math
import datetime
import csv
import statistics
import copy
from collections import defaultdict
from statistics import harmonic_mean

start = time.time()

def calculate_mean(numbers):
    return sum(numbers) / len(numbers)

def calculate_standard_deviation(numbers):
    return statistics.stdev(numbers)

def combine_lists(full_list):
    combined = defaultdict(list)
    
    # 모든 리스트에 대한 처리
    for sublist in full_list:
        for item in sublist:
            entrez_id, rwr_score = item
            combined[entrez_id].append(rwr_score)
    
    # 조화평균 및 결과 리스트 만들기
    result = []
    for entrez_id, scores in combined.items():
        if len(scores) == 1:
            result.append([entrez_id, scores[0]])  # 값이 하나면 그대로 추가
        else:
            # 조화평균 계산
            harmonic_mean_score = harmonic_mean(scores)
            result.append([entrez_id, harmonic_mean_score])
    
    return result

def create_ppi_network(network_data):
    network_data = network_data.astype({'Entrez Gene Interactor A': str,
                                        'Entrez Gene Interactor B': str})

    # symbol list를 생성하고 unique 값만 가지도록 생성
    symbolA = network_data.loc[:, 'Entrez Gene Interactor A'].to_list()
    symbolB = network_data.loc[:, 'Entrez Gene Interactor B'].to_list()

    symbol_list = symbolA + symbolB 
    symbol_list = set(symbol_list) 
    symbol_list = list(symbol_list)

    # 모든 symbol에 index를 딕셔너리에서 부여
    node_index = {}
    for i in range(len(symbol_list)):
        node_index[symbol_list[i]] = i

    # index로 node_list 생성
    node_list = node_index.values()
    
    # index로 edge_list 생성
    edge_list = network_data[['Entrez Gene Interactor A', 'Entrez Gene Interactor B']].values.tolist()  #
    for i in range(len(edge_list)):
        edge_list[i][0] = node_index.get(edge_list[i][0])
        edge_list[i][1] = node_index.get(edge_list[i][1])
    
    # 무방향 그래프 오브젝트 생성
    G = nx.Graph()

    # 그래프에 노드 추가
    G.add_nodes_from(node_list)

    # 그래프에 엣지 추가
    G.add_edges_from(edge_list)

    print(f'Number of created nodes: {G.number_of_nodes()}')
    print(f'Number of created edges: {G.number_of_edges()}')

    return G, node_index

# postgresql DB 연동
config = configparser.ConfigParser()
config.read('db_config.ini')
conn=psycopg2.connect(host="168.131.30.66", dbname="coconut",user="dbuser",password="jnudl1") #데이터이름변경
conn_cur = conn.cursor()

# BIOGRID-Homo_sapiens Data 기반 PPI Network 구축
network_data = pd.read_csv("/data/home/sss2061/식약처/Data/BIOGRID-ORGANISM-Homo_sapiens.tab3.txt", sep="\t", encoding="cp949")  
G, node_index = create_ppi_network(network_data=network_data)  
entrez_list = list(node_index.keys())
entrez_list.remove('-')
gene_phenotype_file = '/data/home/sss2061/식약처보고서/서울과기대/Data/HPO_gene_phenotype.csv'
gene_phenotype_df= pd.read_csv(gene_phenotype_file, encoding='UTF-8', converters={'Phenotype': ast.literal_eval})

/data/home/bmi-lab/anaconda3/envs/jy1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (1,2,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Number of created nodes: 27421
Number of created edges: 817463


In [8]:
# 각 phenotype에 대한 Entrez ID들의 리스트를 만들기 위한 빈 딕셔너리
phenotype_entrez_dict = {}

#------------- 1차 추가
number=1
text=["adenine", "canavanin", "malate", "isoleucine", "phenylalanine", "tyrosine", "isocitrate", "glutamine", "tryptophan","L-malate", 
      "theogallin", "trans-aconitate", "cis-aconitate", "threonate", "benzoic acid", "choline", "citric acid", "coumarin", "phloroglucinol", "gallic acid", 
      "citrullin", "pipecolic acid", "vitamin B", "pantothenic acid", "DL-valine", "vanillin", "theophylline", "caffeine", "protocatechuic aldehyde", "adenosine", 
      "EGCG", "gallocatechin", "N-acetylglutamate", "loliolide", "procyanidin B2", "3H-proline", "dihydromyricetin", "beta-alanine betaine", "epicatechin gallate (ECG", "5'-methylthioadenosine", 
      "epiafzelechin", "ferulic acid", "p-coumaric acid", "caffeic acid", "chlorogenic acid", "quercetin", "alpha-isopropylmalate", "neochlorogenic acid", "rutin", "hyperoside", 
      "4-p-coumaroylquinic acid", "astragalin", "quercetin 7-O-glucoside", "p-coumaroylquinic acid", "procyanidin B1"]

Compoundid_list=[]
for compound in text:
    # SQL 쿼리에서 따옴표 이스케이프
#     compound_escaped = compound.replace("'", "''")
    conn_cur.execute("SELECT compid FROM compound WHERE name = %s", (compound,))
    result = conn_cur.fetchone()
    if result:
        Compoundid_list.append(result[0])
    else:
        Compoundid_list.append(None)  # 데이터가 없는 경우 None 추가

target_gene_list = []
indirected_target_gene_list = [] 
all_target_gene_list=[]
print(Compoundid_list) # 제거가능

for compoundid in Compoundid_list:
    
    conn_cur.execute(f"with tmp1 as (select cg.geneid from compound_gene cg where (cg.compid = '{compoundid}' and cg.relationsign in ('activate','inhibit')))" 
                    f"select g.entrezid from gene g,tmp1 where g.geneid=tmp1.geneid;")
    compound_gene1=conn_cur.fetchall()

    compound_gene=set(compound_gene1)
    compound_gene=list(compound_gene)
    # Compound-target gene List 생성
    for i in range(len(compound_gene)):
        interaction_gene = compound_gene[i][0]
        if interaction_gene in entrez_list:
            target_gene_list.append(interaction_gene)        
        else:
            continue
            
        
    conn_cur.execute(f"with tmp2 as (select cg.geneid from compound_gene cg where cg.compid = '{compoundid}' and relationsign='associate')" 
                    f"select g.entrezid from gene g,tmp2 where g.geneid=tmp2.geneid;")

    compound_gene2=conn_cur.fetchall()

    compound_gene=set(compound_gene2)
    compound_gene=list(compound_gene)

    for i in range(len(compound_gene)):
        interaction_gene = compound_gene[i][0]
        if interaction_gene in entrez_list:
            indirected_target_gene_list.append(interaction_gene)        
        else:
            continue    

    indirected_target_gene_list = [gene for gene in indirected_target_gene_list if gene not in target_gene_list]
    all_target_gene_list=all_target_gene_list+target_gene_list + indirected_target_gene_list

all_target_gene_list=set(all_target_gene_list)
all_target_gene_list=list(all_target_gene_list)

#리스트 내 요소 문자형 컴마 -> 정수형으로 변경
all_target_gene_list = [int(item) for item in all_target_gene_list]
# 각 행에 대해 반복
for index, row in gene_phenotype_df.iterrows():
    entrez_id = row['Entrez ID']
    if entrez_id in all_target_gene_list:
        for phenotype in row['Phenotype']:
            if phenotype in phenotype_entrez_dict:
                phenotype_entrez_dict[phenotype].add(entrez_id)
            else:
                phenotype_entrez_dict[phenotype] = {entrez_id}

# 딕셔너리에서 데이터프레임으로 변환
final_df = pd.DataFrame([(key, list(values)) for key, values in phenotype_entrez_dict.items()], 
                        columns=['Phenotype', 'Entrez ID'])
final_df.to_csv('result/{}차/{}_Seed node_phenotype.csv'.format(number,number),index=False, encoding='UTF-8')

['CP00000181', 'CP00000259', 'CP00000492', 'CP00000740', 'CP00000929', 'CP00001073', 'CP00001113', 'CP00005410', 'CP00005657', 'CP00176744', 'CP00369758', 'CP00000289', 'CP00547717', 'CP04590769', 'CP00000230', 'CP00000285', 'CP00000291', 'CP00000302', 'CP00000338', 'CP00000347', 'CP00000779', 'CP00000795', 'CP00000874', 'CP00000923', 'CP00001100', 'CP00001101', 'CP00001981', 'CP00002311', 'CP00007903', 'CP00046861', 'CP00049992', 'CP00050011', 'CP00000176', 'CP00080459', 'CP00097591', 'CP00117139', 'CP00129832', 'CP00154678', 'CP00307121', 'CP00366769', 'CP00370251', 'CP00000663', 'CP00000301', 'CP00002310', 'CP01484737', 'CP04449797', 'CP04449895', 'CP04449973', 'CP04450096', 'CP04450663', 'CP04450778', 'CP04451007', 'CP04451041', 'CP05102274', 'CP09137603']
